## Keras キャッチアップ

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [0]:
model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, input_shape=(2,))])

# ANDゲートの学習データを用意
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.1),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=100,
                    verbose=1)

y_pred_proba = model.predict(x_train)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Epoch 1/100
4/4 [==============================] - 0s 37ms/sample - loss: 0.8081 - acc: 0.5000
Epoch 2/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.7300 - acc: 0.5000
Epoch 3/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.7188 - acc: 0.5000
Epoch 4/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.6694 - acc: 0.5000
Epoch 5/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.6322 - acc: 0.5000
Epoch 6/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.6075 - acc: 0.5000
Epoch 7/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.5882 - acc: 0.5000
Epoch 8/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.5714 - acc: 0.7500
Epoch 9/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.5459 - acc: 0.7500
Epoch 10/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.5208 - acc: 0.7500
Epoch 11/100
4/4 [==============================] - 0s 2ms

In [46]:
input_data = tf.keras.layers.Input(shape=(2,))
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(10, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=input_data, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.1),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=1,
                    epochs=100,
                    verbose=1)

y_pred_proba = model.predict(x_train)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)

Epoch 1/100
4/4 [==============================] - 0s 58ms/sample - loss: 1.5607 - acc: 0.7500
Epoch 2/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.7878 - acc: 0.7500
Epoch 3/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.6584 - acc: 0.7500
Epoch 4/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.6239 - acc: 0.7500
Epoch 5/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.5812 - acc: 0.7500
Epoch 6/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.5805 - acc: 0.7500
Epoch 7/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.5716 - acc: 0.7500
Epoch 8/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.5715 - acc: 0.7500
Epoch 9/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.5781 - acc: 0.7500
Epoch 10/100
4/4 [==============================] - 0s 3ms/sample - loss: 0.5718 - acc: 0.7500
Epoch 11/100
4/4 [==============================] - 0s 3ms

## 【問題1】公式Exampleを分担して実行
### custom layers
- tf.keras はニューラルネットワークを構築するための、高レベルAPIとしての使用が推奨されている
- ほとんどのtensor flow APIはenable_eager_execution()で使用は可能

In [0]:
# most TensorFlow APIs are usable with eager execution
#tf.enable_eager_execution()

- tensor flow では単純なレイヤーをスタッキングして表現できる
- 多くの一般的なレイヤー(Conv2D, LSTM...)と独自のレイヤーを組み合わせて使用することができる

In [0]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

- レイヤの構成はlayer()で見れる
- layer.variables, kernel, bias などで各項目に注目して見れる

In [49]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))
#layer.variables
#layer.kernel
#layer.bias

<tf.Tensor 'dense_22/BiasAdd:0' shape=(10, 10) dtype=float32>

レイヤーモジュールたち  
https://www.tensorflow.org/api_docs/python/tf/keras/layers

### custom layersの実装
1. tf.keras.layers.Layerを継承する super()
2. 親クラスの__init__()を実行する
3. self.num_outputsを引数のoutputsで定義する
3. build()を作成し、self.kernelをadd_variableで上書きする
4. call()を作成し、フォワードの処理を記述・処理結果を返り値とする(インスタンスcall時に呼ばれる)

In [50]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[int(input_shape[-1]), 
                                           self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

Tensor("my_dense_layer_3/MatMul:0", shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer_3/kernel:0' shape=(5, 10) dtype=float32>]


### レイヤーモデルを構成する 例:Resnetモデル
1. tf.keras.Modelを継承する
2. 親クラスの__init__()を実行
3. 引数のサイズを元に各レイヤーを構成する
4. call()を作成し、レイヤーごとの処理を記述・処理結果を返り値とする(インスタンスcall時に呼ばれる)

In [51]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

Tensor("resnet_identity_block_2/Relu_2:0", shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block_2/conv2d_12/kernel:0', 'resnet_identity_block_2/conv2d_12/bias:0', 'resnet_identity_block_2/batch_normalization_v1_12/gamma:0', 'resnet_identity_block_2/batch_normalization_v1_12/beta:0', 'resnet_identity_block_2/conv2d_13/kernel:0', 'resnet_identity_block_2/conv2d_13/bias:0', 'resnet_identity_block_2/batch_normalization_v1_13/gamma:0', 'resnet_identity_block_2/batch_normalization_v1_13/beta:0', 'resnet_identity_block_2/conv2d_14/kernel:0', 'resnet_identity_block_2/conv2d_14/bias:0', 'resnet_identity_block_2/batch_normalization_v1_14/gamma:0', 'resnet_identity_block_2/batch_normalization_v1_14/beta:0']


上記はresnetのように構成するレイヤーが決まっている場合に実装する。(テンプレート的に使いたい)  
それ以外の場合は下記のようにtf.keras.Sequentialを使って実行する

In [52]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(2, 1, 
                                                      padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(3, (1, 1)),
                               tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor 'sequential_5/cond_4/Merge:0' shape=(1, 2, 3, 3) dtype=float32>

## 【問題2】Iris（2値分類）をKerasで学習

In [53]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

model = Sequential()
model.add(Dense(12, input_dim=4))
model.add(Activation('relu'))
model.add(Dense(3, input_dim=12))
model.add(Activation('softmax')) 
model.compile(optimizer='SGD',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=20, batch_size=5)

# モデル評価
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
64/64 [==============================] - 1s 12ms/step - loss: 0.6847 - acc: 0.6094
Epoch 2/20
64/64 [==============================] - 0s 313us/step - loss: 0.6566 - acc: 0.6094
Epoch 3/20
64/64 [==============================] - 0s 276us/step - loss: 0.6378 - acc: 0.5938
Epoch 4/20
64/64 [==============================] - 0s 270us/step - loss: 0.6163 - acc: 0.6250
Epoch 5/20
64/64 [==============================] - 0s 290us/step - loss: 0.6247 - acc: 0.6719
Epoch 6/20
64/64 [==============================] - 0s 259us/step - loss: 0.6186 - acc: 0.7344
Epoch 7/20
64/64 [==============================] - 0s 318us/step - loss: 0.6082 - acc: 0.6875
Epoch 8/20
64/64 [==============================] - 0s 322us/step - loss: 0.5552 - acc: 0.7656
Epoch 9/20
64/64 [==============================] - 0s 254us/step - loss: 0.5545 - acc: 0.7344
Epoch 10/20
64/64 [==============================] - 0s 255us/step - loss: 0.5424 - acc: 0.8125
Epoch 11/20
64/64 [==============================]

## 【問題3】Iris（多値分類）をKerasで学習

In [54]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
# ワンホット化
y = to_categorical(y).astype(np.int)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

model = Sequential()
model.add(Dense(12, input_dim=4))
model.add(Activation('relu'))
model.add(Dense(3, input_dim=12))
model.add(Activation('softmax')) 
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=20, batch_size=5)

# モデル評価
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
96/96 [==============================] - 1s 8ms/step - loss: 1.2491 - acc: 0.5313
Epoch 2/20
96/96 [==============================] - 0s 279us/step - loss: 0.8502 - acc: 0.6354
Epoch 3/20
96/96 [==============================] - 0s 261us/step - loss: 0.6826 - acc: 0.7292
Epoch 4/20
96/96 [==============================] - 0s 263us/step - loss: 0.6390 - acc: 0.6667
Epoch 5/20
96/96 [==============================] - 0s 255us/step - loss: 0.5798 - acc: 0.6979
Epoch 6/20
96/96 [==============================] - 0s 282us/step - loss: 0.5270 - acc: 0.7813
Epoch 7/20
96/96 [==============================] - 0s 307us/step - loss: 0.5014 - acc: 0.7917
Epoch 8/20
96/96 [==============================] - 0s 361us/step - loss: 0.5150 - acc: 0.6875
Epoch 9/20
96/96 [==============================] - 0s 287us/step - loss: 0.4722 - acc: 0.8229
Epoch 10/20
96/96 [==============================] - 0s 266us/step - loss: 0.4303 - acc: 0.8125
Epoch 11/20
96/96 [==============================] 

## 【問題4】House PricesをKerasで学習

In [55]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
y = y[:, np.newaxis]

scaler = StandardScaler()
scaler.fit(y)
y = scaler.transform(y)

X = np.array(X)


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

model = Sequential()
model.add(Dense(3, input_dim=2))
model.add(Activation('relu'))
model.add(Dense(1, input_dim=3))
model.add(Activation(None)) 
model.compile(optimizer='SGD',loss='mean_squared_error',metrics=['mean_squared_error'])

model.fit(X_train, y_train, nb_epoch=20, batch_size=5)

# モデル評価
loss, mean_squared_error = model.evaluate(X_val, y_val, verbose=0)
print("mean_squared_error = {:.2f}".format(mean_squared_error))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/20
934/934 [==============================] - 1s 1ms/step - loss: 0.8939 - mean_squared_error: 0.8939
Epoch 2/20
934/934 [==============================] - 0s 186us/step - loss: 0.5316 - mean_squared_error: 0.5316
Epoch 3/20
934/934 [==============================] - 0s 186us/step - loss: 0.4118 - mean_squared_error: 0.4118
Epoch 4/20
934/934 [==============================] - 0s 195us/step - loss: 0.3423 - mean_squared_error: 0.3423
Epoch 5/20
934/934 [==============================] - 0s 187us/step - loss: 0.3188 - mean_squared_error: 0.3188
Epoch 6/20
934/934 [==============================] - 0s 193us/step - loss: 0.3298 - mean_squared_error: 0.3298
Epoch 7/20
934/934 [==============================] - 0s 193us/step - loss: 0.3159 - mean_squared_error: 0.3159
Epoch 8/20
934/934 [==============================] - 0s 190us/step - loss: 0.3245 - mean_squared_error: 0.3245
Epoch 9/20
934/934 [==============================] - 0s 198us/step - loss: 0.3209 - mean_squared_error: 0

## 【問題5】MNISTのモデルを作成

In [57]:
(X_train, y_train), (_, _) = mnist.load_data()

# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

X_train = X_train[:, :, :, np.newaxis]

# ワンホット化
y_train = to_categorical(y_train).astype(np.int)


# 学習データをスプリット
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

model = Sequential()
model.add(Conv2D(120, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(84, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Activation('softmax')) 

model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=3, batch_size=5)

# モデル評価
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/3
38400/38400 [==============================] - 214s 6ms/step - loss: 1.2808 - acc: 0.5058
Epoch 2/3
38400/38400 [==============================] - 212s 6ms/step - loss: 0.3101 - acc: 0.8813
Epoch 3/3
38400/38400 [==============================] - 212s 6ms/step - loss: 0.0605 - acc: 0.9815
Accuracy = 0.98
